In [13]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

In [14]:
# Load CSV dataset
df = pd.read_csv("C:/Users/singh/OneDrive/Desktop/Vaish/Reviews.csv")

# Quick look at data
display(df.head())
display(df.info())
display(df.isnull().sum())

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


None

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [15]:
df = df[['Score', 'Summary', 'Text']]

# Handle null values
df['Summary'] = df['Summary'].fillna('')

# Combine into one column
df['review'] = df['Summary'] + " " + df['Text']

display(df.head(5))


,Score,Summary,Text,review
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Good Quality Dog Food I have bought several of...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised Product arrived labeled as J...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...,"""Delight"" says it all This is a confection tha..."
3,2,Cough Medicine,If you are looking for the secret ingredient i...,Cough Medicine If you are looking for the secr...
4,5,Great taffy,Great taffy at a great price. There was a wid...,Great taffy Great taffy at a great price. The...


In [16]:
#Preprocessing Text
#Raw Review:
#"This product is AMAZING!!! I’ve been using it for 2 months & it works better than others."

#After Preprocessing:
#"product amazing month work good"

#Remove stopwords, stopwords are common words (like the, is, was) that add little meaning.
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    
    # Lower case all text so everything is same format
    text = text.lower()

    #remove punctuation , numbers and special charachters
    text = re.sub(r'[^a-z\s]', '', text)   
    words = text.split()

    # Normalize words to their base form like running will become run , better will be good, mice will be mouse
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

df['clean_review'] = df['review'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
display(df.head(5))

,Score,Summary,Text,review,clean_review
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Good Quality Dog Food I have bought several of...,good quality dog food bought several vitality ...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised Product arrived labeled as J...,advertised product arrived labeled jumbo salte...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...,"""Delight"" says it all This is a confection tha...",delight say confection around century light pi...
3,2,Cough Medicine,If you are looking for the secret ingredient i...,Cough Medicine If you are looking for the secr...,cough medicine looking secret ingredient robit...
4,5,Great taffy,Great taffy at a great price. There was a wid...,Great taffy Great taffy at a great price. The...,great taffy great taffy great price wide assor...


In [19]:
#Sentiment Analysis based on Score column

def label_sentiment(score):
    if score >= 4:
        return 'Positive'
    elif score == 3:
        return 'Neutral'
    else:
        return 'Negative'

df['sentiment'] = df['Score'].apply(label_sentiment)

display(df.head(5))


,Score,Summary,Text,review,clean_review,sentiment
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Good Quality Dog Food I have bought several of...,good quality dog food bought several vitality ...,Positive
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised Product arrived labeled as J...,advertised product arrived labeled jumbo salte...,Negative
2,4,"""Delight"" says it all",This is a confection that has been around a fe...,"""Delight"" says it all This is a confection tha...",delight say confection around century light pi...,Positive
3,2,Cough Medicine,If you are looking for the secret ingredient i...,Cough Medicine If you are looking for the secr...,cough medicine looking secret ingredient robit...,Negative
4,5,Great taffy,Great taffy at a great price. There was a wid...,Great taffy Great taffy at a great price. The...,great taffy great taffy great price wide assor...,Positive
